In [1]:
# Cell 1: Imports and Introduction
import pandas as pd # Used for better display of DCF details

# ==============================================================================
# HEADER: INTRODUCTION AND INSTRUCTIONS
# ==============================================================================
print("="*70)
print(" Simplified Credit Analysis & Valuation Notebook")
print("="*70)
print("""
This notebook provides a highly simplified framework for:
1. Credit Rating and Probability of Default (PD) Estimation
2. Discounted Cash Flow (DCF) Analysis
3. Enterprise Value (EV) from Multiples Analysis

Please provide the requested inputs in the cells below.
The analyses are illustrative and use simplified assumptions.
This notebook is for educational purposes and not for making financial decisions.
""")

# Cell 2: User Inputs
# ==============================================================================
# SECTION 1: USER INPUTS
# ==============================================================================
print("\n" + "="*70)
print(" SECTION 1: USER INPUTS")
print("="*70)

# --- Inputs for PD Model ---
print("\n--- 1.1 Inputs for Simplified PD Model ---")
company_name = ""
debt_to_equity = 0.0
interest_coverage_ratio = 0.0
management_quality_score = 0
industry_outlook_score = 0

try:
    company_name = input("Enter Company Name: ")
    if not company_name.strip():
        company_name = "Test Company Inc."
        print(f"  No company name entered, using default: {company_name}")

    debt_to_equity_str = input("Enter Debt-to-Equity Ratio (e.g., 0.5 for 50%): ")
    debt_to_equity = float(debt_to_equity_str) if debt_to_equity_str else 0.5
    if not debt_to_equity_str: print(f"  Using default D/E: {debt_to_equity}")


    interest_coverage_ratio_str = input("Enter Interest Coverage Ratio (e.g., 3.5): ")
    interest_coverage_ratio = float(interest_coverage_ratio_str) if interest_coverage_ratio_str else 3.5
    if not interest_coverage_ratio_str: print(f"  Using default ICR: {interest_coverage_ratio}")

    # Example qualitative scores: 1 (Poor), 2 (Average), 3 (Good), 4 (Excellent)
    management_quality_score_str = input("Enter Management Quality Score (1-4, where 4 is best): ")
    management_quality_score = int(management_quality_score_str) if management_quality_score_str else 3
    if not management_quality_score_str: print(f"  Using default Mgmt Score: {management_quality_score}")
    if not 1 <= management_quality_score <= 4:
        print("  Management quality score out of range (1-4). Setting to 2 (Average).")
        management_quality_score = 2


    industry_outlook_score_str = input("Enter Industry Outlook Score (1-4, where 4 is best): ")
    industry_outlook_score = int(industry_outlook_score_str) if industry_outlook_score_str else 3
    if not industry_outlook_score_str: print(f"  Using default Industry Score: {industry_outlook_score}")
    if not 1 <= industry_outlook_score <= 4:
        print("  Industry outlook score out of range (1-4). Setting to 2 (Average).")
        industry_outlook_score = 2

except ValueError:
    print("Invalid input detected. Using default values for PD model inputs.")
    company_name = company_name if company_name else "Default Corp"
    debt_to_equity = 0.5
    interest_coverage_ratio = 3.5
    management_quality_score = 2
    industry_outlook_score = 2

# --- Inputs for DCF Analysis ---
print("\n--- 1.2 Inputs for Simplified DCF Analysis ---")
num_projection_years = 3 # Simplified to 3 years
projected_fcfs = []
wacc = 0.0
terminal_growth_rate = 0.0

print(f"Enter projected Free Cash Flows (FCF) for the next {num_projection_years} years:")
default_fcfs = [100, 110, 120] # Default FCFs if none entered
for i in range(num_projection_years):
    while True:
        try:
            fc_val_str = input(f"  FCF Year {i+1} (default: {default_fcfs[i]}): ")
            fc_val = float(fc_val_str) if fc_val_str else default_fcfs[i]
            projected_fcfs.append(fc_val)
            if not fc_val_str: print(f"    Using default FCF Year {i+1}: {fc_val}")
            break
        except ValueError:
            print("    Invalid input. Please enter a numeric value for FCF.")

while True:
    try:
        wacc_str = input("Enter Discount Rate (WACC) (e.g., 0.10 for 10%, default: 0.10): ")
        wacc = float(wacc_str) if wacc_str else 0.10
        if not wacc_str: print(f"  Using default WACC: {wacc}")
        if 0 < wacc < 1:
            break
        else:
            print("    WACC should be between 0 and 1 (e.g., 0.1 for 10%). Please re-enter or use default.")
            if not wacc_str: wacc = 0.10; break # Allow default to pass
    except ValueError:
        print("    Invalid input. Please enter a numeric value for WACC.")

while True:
    try:
        terminal_growth_rate_str = input("Enter Terminal Growth Rate (e.g., 0.02 for 2%, default: 0.02): ")
        terminal_growth_rate = float(terminal_growth_rate_str) if terminal_growth_rate_str else 0.02
        if not terminal_growth_rate_str: print(f"  Using default Terminal Growth: {terminal_growth_rate}")

        if 0 <= terminal_growth_rate < wacc : # Terminal growth should ideally be less than WACC
            break
        else:
            print(f"    Terminal growth rate ({terminal_growth_rate:.2%}) should be positive and less than WACC ({wacc:.2%}).")
            if not terminal_growth_rate_str: # If default was used and it's invalid, adjust WACC or TGR
                if terminal_growth_rate >= wacc:
                    print(f"    Adjusting default terminal growth to be less than WACC. New TGR: {wacc - 0.01:.2%}")
                    terminal_growth_rate = wacc - 0.01
                    if terminal_growth_rate < 0: terminal_growth_rate = 0.005 # Ensure non-negative
            # If user input is invalid, loop again.
            elif terminal_growth_rate >= wacc :
                 continue # Force re-entry if user provided invalid TGR
            break # Break if adjusted default is valid or user input is valid

    except ValueError:
        print("    Invalid input. Please enter a numeric value for terminal growth rate.")


# --- Inputs for EV (Multiples) Analysis ---
print("\n--- 1.3 Inputs for Simplified EV (Multiples) Analysis ---")
relevant_metric_name = "EBITDA"
relevant_metric_value = 0.0
industry_multiple = 0.0

while True:
    try:
        metric_name_str = input(f"Enter name of the financial metric for EV multiple (default: {relevant_metric_name}): ")
        relevant_metric_name = metric_name_str if metric_name_str.strip() else relevant_metric_name
        if not metric_name_str.strip(): print(f"  Using default metric name: {relevant_metric_name}")

        metric_value_str = input(f"Enter value of {relevant_metric_name} (default: 200): ")
        relevant_metric_value = float(metric_value_str) if metric_value_str else 200
        if not metric_value_str: print(f"  Using default metric value: {relevant_metric_value}")

        industry_multiple_str = input(f"Enter industry average multiple for {relevant_metric_name} (default: 8.0): ")
        industry_multiple = float(industry_multiple_str) if industry_multiple_str else 8.0
        if not industry_multiple_str: print(f"  Using default industry multiple: {industry_multiple}")
        break
    except ValueError:
        print("    Invalid input. Please enter numeric values for metric value and multiple.")

# Cell 3: Simplified Credit Rating & PD Model
# ==============================================================================
# SECTION 2: SIMPLIFIED CREDIT RATING & PROBABILITY OF DEFAULT (PD)
# ==============================================================================
print("\n" + "="*70)
print(f" SECTION 2: CREDIT RATING & PD FOR - {company_name.upper()}")
print("="*70)

# --- 2.1 Simplified Scoring Logic ---
# This scoring logic is purely illustrative and highly simplified.
# Real credit scoring models are statistically derived and much more complex.

# Score contributions (higher score means better credit quality)
score_d2e = 0
if debt_to_equity < 0.25: score_d2e = 3  # Very Low D/E
elif debt_to_equity < 0.75: score_d2e = 2 # Low D/E
elif debt_to_equity < 1.5: score_d2e = 1  # Moderate D/E
elif debt_to_equity < 2.5: score_d2e = 0  # High D/E
else: score_d2e = -1 # Very High D/E

score_icr = 0
if interest_coverage_ratio > 8: score_icr = 3  # Excellent ICR
elif interest_coverage_ratio > 5: score_icr = 2  # Good ICR
elif interest_coverage_ratio > 2.5: score_icr = 1  # Fair ICR
elif interest_coverage_ratio > 1: score_icr = 0    # Poor ICR
else: score_icr = -1 # Very Poor ICR / Default Risk

# Qualitative scores (1-4 scale) are mapped to a contribution.
# Example: 4 -> +2, 3 -> +1, 2 -> 0, 1 -> -1
score_mgmt = management_quality_score - 2
score_ind = industry_outlook_score - 2

total_score = score_d2e + score_icr + score_mgmt + score_ind

# --- 2.2 Ratings Probability Map ---
# This map is purely illustrative. Score range could be approx -1 + -1 + -1 + -1 = -4 to 3 + 3 + 2 + 2 = 10
credit_rating = "Undefined"
probability_of_default = 0.0
pd_justification_factors = []

if total_score >= 7:
    credit_rating = "AAA (Prime)"
    probability_of_default = 0.001 # 0.1%
elif total_score >= 5:
    credit_rating = "AA (High Grade)"
    probability_of_default = 0.005 # 0.5%
elif total_score >= 3:
    credit_rating = "A (Upper Medium)"
    probability_of_default = 0.015 # 1.5%
elif total_score >= 1:
    credit_rating = "BBB (Lower Medium)"
    probability_of_default = 0.04 # 4%
elif total_score >= -1:
    credit_rating = "BB (Non-Investment Grade - Speculative)"
    probability_of_default = 0.10 # 10%
elif total_score >= -3:
    credit_rating = "B (Highly Speculative)"
    probability_of_default = 0.20 # 20%
else: # total_score < -3
    credit_rating = "CCC/C (Substantial Risk / Default Imminent)"
    probability_of_default = 0.35 # 35%+

# --- 2.3 Output and Justification ---
print(f"\n--- PD Model Inputs Summary & Score Contributions ---")
print(f"  Debt-to-Equity Ratio: {debt_to_equity:.2f} (Score Contribution: {score_d2e})")
pd_justification_factors.append(f"Debt-to-Equity of {debt_to_equity:.2f} contributed {score_d2e} to the score.")

print(f"  Interest Coverage Ratio: {interest_coverage_ratio:.2f} (Score Contribution: {score_icr})")
pd_justification_factors.append(f"Interest Coverage Ratio of {interest_coverage_ratio:.2f} contributed {score_icr} to the score.")

print(f"  Management Quality Score: {management_quality_score}/4 (Score Contribution: {score_mgmt})")
pd_justification_factors.append(f"Management Quality score of {management_quality_score}/4 contributed {score_mgmt} to the score.")

print(f"  Industry Outlook Score: {industry_outlook_score}/4 (Score Contribution: {score_ind})")
pd_justification_factors.append(f"Industry Outlook score of {industry_outlook_score}/4 contributed {score_ind} to the score.")

print(f"\n--- PD Model Results ---")
print(f"  Total Calculated Score: {total_score}")
print(f"  Assigned Credit Rating: {credit_rating}")
print(f"  Estimated Probability of Default (PD): {probability_of_default:.2%}")

print("\n--- Justification for PD & Rating ---")
print("The credit rating and PD are derived from a simplified scoring model based on your inputs:")
for factor in pd_justification_factors:
    print(f"  - {factor}")
print(f"\nThe sum of these contributions gives a total score of {total_score}.")
print(f"Based on our illustrative rating map, this score corresponds to a '{credit_rating}' rating,")
print(f"which has an associated illustrative Probability of Default of {probability_of_default:.2%}.")
print("\nDisclaimer: This is a rudimentary model for illustrative purposes and not a substitute for professional credit assessment.")


# Cell 4: Simplified DCF Analysis
# ==============================================================================
# SECTION 3: SIMPLIFIED DISCOUNTED CASH FLOW (DCF) ANALYSIS
# ==============================================================================
print("\n" + "="*70)
print(f" SECTION 3: DCF ANALYSIS FOR - {company_name.upper()}")
print("="*70)

pv_fcf_total = 0
dcf_details_list = [] # Changed name to avoid conflict

print("\n--- DCF Calculation Details ---")
# PV of projected FCFs
for i in range(num_projection_years):
    discount_factor = (1 + wacc) ** (i + 1)
    pv_fcf = projected_fcfs[i] / discount_factor
    pv_fcf_total += pv_fcf
    dcf_details_list.append({
        "Year": i + 1,
        "Projected FCF": f"{projected_fcfs[i]:,.2f}",
        "Discount Factor": f"{discount_factor:.4f}",
        "PV of FCF": f"{pv_fcf:,.2f}"
    })

# Terminal Value (Gordon Growth Model)
# FCF_last_projected_year * (1 + terminal_growth_rate) / (wacc - terminal_growth_rate)
if (wacc - terminal_growth_rate) <= 0:
    print("\nError: WACC must be greater than Terminal Growth Rate for Terminal Value calculation.")
    print("Cannot calculate Terminal Value or Total Firm Value via DCF with current inputs.")
    terminal_value = 0
    pv_terminal_value = 0
    total_firm_value_dcf = pv_fcf_total # Only sum of projected PVs
else:
    last_projected_fcf = projected_fcfs[-1]
    terminal_value_numerator = last_projected_fcf * (1 + terminal_growth_rate)
    terminal_value_denominator = (wacc - terminal_growth_rate)
    terminal_value = terminal_value_numerator / terminal_value_denominator
    pv_terminal_value = terminal_value / ((1 + wacc) ** num_projection_years)
    total_firm_value_dcf = pv_fcf_total + pv_terminal_value


dcf_summary_df = pd.DataFrame(dcf_details_list)
print("Projected Period Cash Flows:")
print(dcf_summary_df.to_string(index=False))

print(f"\n  Sum of PV of Projected FCFs (Years 1-{num_projection_years}): {pv_fcf_total:,.2f}")

if (wacc - terminal_growth_rate) > 0:
    print(f"  Terminal Value (calculated at end of Year {num_projection_years}): {terminal_value:,.2f}")
    print(f"  Present Value of Terminal Value: {pv_terminal_value:,.2f}")
    print(f"\n  Implied Total Firm Value (DCF): {total_firm_value_dcf:,.2f}")
else:
    print(f"\n  Implied Total Firm Value (DCF - excluding Terminal Value): {total_firm_value_dcf:,.2f}")


print("\nNote: This DCF is highly simplified. It assumes FCFs are Free Cash Flow to the Firm (FCFF).")
print("To derive Equity Value, you would typically subtract Net Debt and add Cash & Cash Equivalents from/to Total Firm Value.")


# Cell 5: Simplified EV (Multiples) Analysis
# ==============================================================================
# SECTION 4: SIMPLIFIED ENTERPRISE VALUE (EV) - MULTIPLES
# ==============================================================================
print("\n" + "="*70)
print(f" SECTION 4: EV (MULTIPLES) ANALYSIS FOR - {company_name.upper()}")
print("="*70)

enterprise_value_multiples = relevant_metric_value * industry_multiple

print(f"\n--- EV (Multiples) Calculation ---")
print(f"  Financial Metric Used: {relevant_metric_name}")
print(f"  Value of {relevant_metric_name}: {relevant_metric_value:,.2f}")
print(f"  Assumed Industry Multiple for {relevant_metric_name}: {industry_multiple:.1f}x")
print(f"\n  Calculated Enterprise Value (EV) via Multiples: {enterprise_value_multiples:,.2f}")
print("\nNote: The accuracy of this EV calculation depends heavily on the appropriateness")
print("of the chosen financial metric and the comparability of the industry multiple selected.")
print("This is a common 'sanity check' or relative valuation method.")

# Cell 6: Summary
# ==============================================================================
# SECTION 5: ANALYSIS SUMMARY
# ==============================================================================
print("\n" + "="*70)
print(" SECTION 5: ANALYSIS SUMMARY & CONCLUSION")
print("="*70)

print(f"Company Analyzed: {company_name.upper()}")
print(f"\n  Simplified Credit Assessment:")
print(f"    - Estimated Credit Rating: {credit_rating}")
print(f"    - Estimated Probability of Default (PD): {probability_of_default:.2%}")

print(f"\n  Simplified Valuation Outputs:")
if (wacc - terminal_growth_rate) > 0:
    print(f"    - Implied Total Firm Value (DCF): {total_firm_value_dcf:,.2f}")
else:
    print(f"    - Implied Total Firm Value (DCF - TV not calc.): {total_firm_value_dcf:,.2f}")
print(f"    - Calculated Enterprise Value (Multiples - using {relevant_metric_name}): {enterprise_value_multiples:,.2f}")

print("\n" + "-"*70)
print("Important Disclaimer:")
print("The analyses presented in this notebook are based on highly simplified models")
print("and user-provided inputs or illustrative default values. They are intended for")
print("educational and conceptual purposes ONLY. They do NOT constitute financial advice,")
print("credit assessment, or investment recommendations. Always consult with qualified")
print("financial professionals before making any financial decisions.")
print("The 'deep analysis' aspect is reflected in the structured approach and justification")
print("of outputs based on defined logic, not in the complexity of the underlying models.")
print("="*70)

 Simplified Credit Analysis & Valuation Notebook

This notebook provides a highly simplified framework for:
1. Credit Rating and Probability of Default (PD) Estimation
2. Discounted Cash Flow (DCF) Analysis
3. Enterprise Value (EV) from Multiples Analysis

Please provide the requested inputs in the cells below.
The analyses are illustrative and use simplified assumptions.
This notebook is for educational purposes and not for making financial decisions.


 SECTION 1: USER INPUTS

--- 1.1 Inputs for Simplified PD Model ---
Enter Company Name: DKNG
Enter Debt-to-Equity Ratio (e.g., 0.5 for 50%): .4
Enter Interest Coverage Ratio (e.g., 3.5): -2
Enter Management Quality Score (1-4, where 4 is best): 2
Enter Industry Outlook Score (1-4, where 4 is best): 3

--- 1.2 Inputs for Simplified DCF Analysis ---
Enter projected Free Cash Flows (FCF) for the next 3 years:
  FCF Year 1 (default: 100): 100
  FCF Year 2 (default: 110): 100
  FCF Year 3 (default: 120): 100
Enter Discount Rate (WACC) (e.g.